In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
print(sys.path)

Mounted at /content/drive
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Colab Notebooks']


In [2]:
pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [3]:
!pip install transformers accelerate auto-gptq autoawq optimum auto-gptq onnx onnxruntime-gpu==1.17 torchao optimum datasets==2.14.7 zstandard==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 M

In [4]:
import torch
from torchao.quantization import quant_api
from sklearn.metrics import accuracy_score
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
import time
from torch.nn.utils import prune
import pandas as pd

## Base Model

In [5]:
dataset = load_dataset("heegyu/augesc")
label_map = {
    "Question":0,
    "Restatement or Paraphrasing":1,
    "Reflection of feelings":2,
    "Self-disclosure":3,
    "Affirmation and Reassurance":4,
    "Providing Suggestions":5,
    "Information":6,
    "Others":7
}
x = []
y_true = []
for sample in dataset['test']:
    for row in sample['dialog']:
        text = row['text']
        label = row['strategy']
        if label != None:
            x.append(text)
            y_true.append(label_map[label])
x = x[0:1000]
y_true = y_true[0:1000]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
model_id = "heegyu/TinyLlama-augesc-context"
tokenizer = AutoTokenizer.from_pretrained("heegyu/TinyLlama-augesc-context")
model = AutoModelForSequenceClassification.from_pretrained(model_id).eval().to("cuda")
#model = AutoModelForSequenceClassification.from_pretrained(model_id, load_in_8bit=True)
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()
model_size = (param_size + buffer_size) / 1024**2
print('Base Model size: {:.3f}MB'.format(model_size))
y_pred = []
times = []
for current_x, current_y in zip(x, y_true):
    inputs = tokenizer(current_x, return_tensors="pt").to("cuda")
    start_time = time.time()
    logits = model(**inputs).logits.softmax(-1)
    end_time = time.time()
    label = logits.argmax(-1).item()
    y_pred.append(label)
    times.append(end_time - start_time)
print(accuracy_score(y_true, y_pred))
print(pd.Series(times).describe().T)
print(f"Averafe inference time is {sum(times) / len(times)} seconds")

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

Base Model size: 3972.417MB
0.222
count    1000.000000
mean        0.034197
std         0.036204
min         0.029631
25%         0.031761
50%         0.032662
75%         0.033741
max         1.175039
dtype: float64
Averafe inference time is 0.03419733810424805 seconds


## ONNX

In [7]:
import torch
import onnx
import onnxruntime
import os

# Define the path where you want to save your ONNX model
onnx_model_path = '/content/drive/My Drive/Colab Notebooks/onnx/TinyLama_awq/model.onnx'

# Prepare a sample input text for thenput
sample_text = x
# Tokenize the sample text
# Assuming `tokenizer` is already loaded and `tweet` is your current tweet text
input = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True).to('cuda')


# Use the tokenized input as the input for ONNX export
# Assuming your model takes input_ids, attention_mask as inputs
torch.onnx.export(model,
                  args=(inputs['input_ids'], inputs['attention_mask']),
                  f=onnx_model_path,
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['logits'],
                  dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                                'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                                'logits': {0: 'batch_size'}},
                  opset_version=14)

print("Model has been converted to ONNX.")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py:1068: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_length > self.causal_mask.shape[-1]:
/usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py:1380: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.config.pad_token_id is None and batch_size != 1:


Model has been converted to ONNX.


In [19]:
ort.get_device()

'GPU'

In [10]:
import numpy as np
import onnxruntime as ort

# Assuming `texts` is your list of input strings (tweets) and `y_true` are the corresponding labels
texts = x  # Your list of tweets
y_true = y_true  # Your corresponding true labels

# Load the ONNX model with ONNX Runtime
sess_options = ort.SessionOptions()
onnx_model_path = '/content/drive/My Drive/Colab Notebooks/onnx/TinyLama/model.onnx'

# Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization.
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

# To enable model serialization and store the optimized graph to desired location.
sess_options.optimized_model_filepath = onnx_model_path
ort_session = ort.InferenceSession(onnx_model_path, sess_options)
#ort_session = ort.InferenceSession(onnx_model_path, providers=['CUDAExecutionProvider'])

import numpy as np

def softmax(x):
    """
    Compute softmax values for each set of scores in x.
    """
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)


def prepare_single_input(tokenizer, text):
    # Tokenizing a single text
    inputs = tokenizer(text, return_tensors="pt")#tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Convert to numpy arrays on CPU, as ONNX Runtime expects
    #inputs = {k: v.detach().cpu().numpy() for k, v in inputs.items()}
    return inputs

def run_onnx_inference_single(ort_session, tokenizer, texts):
    predictions = []
    times = []
    for text in texts[:200]:
        #text = texts[100]

        inputs = prepare_single_input(tokenizer, text)
        #print(input)
        #input = tokenizer(text, return_tensors="pt")
        onnx_inputs = {'input_ids': np.ascontiguousarray(inputs['input_ids'].cpu().numpy()),
                       'attention_mask': np.ascontiguousarray(inputs['attention_mask'].cpu().numpy())}
        start_time = time.time()
        logits = ort_session.run(None, onnx_inputs)[0]
        end_time = time.time()
        times.append(end_time - start_time)
        #print(logits)
        probs = softmax(logits)

        # Extract the predicted label (class with the highest probability)
        prediction = np.argmax(probs,axis = -1).item()
        predictions.append(prediction)

        #break


    return predictions, times

# Obtain predictions for your inputs, processing each tweet individually
tokenizer = AutoTokenizer.from_pretrained("heegyu/TinyLlama-augesc-context")
predictions, times = run_onnx_inference_single(ort_session, tokenizer, texts)

import os

# # model_size_bytes = os.path.getsize('/content/drive/My Drive/Colab Notebooks/onnx/TinyLama_gptq')
# # model_size_mb = model_size_bytes / (1024 * 1024)

# #predictions now contains the predicted labels for each input text
# print('Base Model size: {:.3f}MB'.format(model_size_mb))
print(accuracy_score(y_true[:200], predictions))
print(pd.Series(times).describe().T)
print(f"Averafe inference time is {sum(times) / len(times)} seconds")

0.195
count    200.000000
mean       0.258362
std        0.115039
min        0.111716
25%        0.175228
50%        0.227727
75%        0.302855
max        0.713327
dtype: float64
Averafe inference time is 0.2583615577220917 seconds


In [10]:
!nvidia-smi

Wed Mar 27 00:26:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              26W /  70W |   5181MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import onnxruntime as ort

# Construct the model path
model_path = os.path.join("/content/drive/My Drive/Colab Notebooks/onnx/TinyLama_gptq/", "model.onnx")

# Create an inference session
ort_session = ort.InferenceSession(model_path)


In [ ]:

def run_onnx_inference_single(ort_session, tokenizer, texts):
    predictions = []
    for text in texts:
        #text = texts[100]
        input = prepare_single_input(tokenizer, text)
        logits = ort_session.run(None, input)[0]

        probs = softmax(logits)

        # Extract the predicted label (class with the highest probability)
        prediction = np.argmax(probs,axis = -1).item()
        predictions.append(prediction)
        #break

    return predictions
predictions = run_onnx_inference_single(ort_session, tokenizer, texts)

In [14]:
accuracy_score(y_true, predictions)

0.222

In [17]:
type(model)

transformers.models.llama.modeling_llama.LlamaForSequenceClassification